Loading stock data:

In [11]:
import pandas as pd

# Load stock returns
df_returns = pd.read_csv("../data/raw/crsp_data/crsp_data.csv")

df_returns['date'] = pd.to_datetime(df_returns['date'])

print(df_returns.shape)
df_returns


(4047630, 3)


,permno,date,ret
0,10000,1986-01-31,NaN
1,10000,1986-02-28,-25.7143
2,10000,1986-03-31,36.5385
3,10000,1986-04-30,-9.8592
4,10000,1986-05-30,-22.2656
...,...,...,...
4047625,93436,2024-08-30,-7.7391
4047626,93436,2024-09-30,22.1942
4047627,93436,2024-10-31,-4.5025
4047628,93436,2024-11-29,38.1469


Load signal score

In [18]:
df_score = pd.read_csv('../data/processed/full_similarity_scores.csv')

df_score['filing_date'] = pd.to_datetime(df_score['filing_date'])

# Preview the data
print(df_score.head())

df_score

    cik filing_date  similarity
0  1750  1994-01-13         NaN
1  1750  1994-04-14    0.991912
2  1750  1994-08-24    0.934598
3  1750  1994-10-14    0.928699
4  1750  1995-01-13    0.868299


,cik,filing_date,similarity
0,1750,1994-01-13,NaN
1,1750,1994-04-14,0.991912
2,1750,1994-08-24,0.934598
3,1750,1994-10-14,0.928699
4,1750,1995-01-13,0.868299
...,...,...,...
1224503,2029303,2024-11-12,NaN
1224504,2031561,2024-11-12,NaN
1224505,2032950,2024-12-26,NaN
1224506,2033615,2024-12-20,NaN


Loading cik-permno map:

In [19]:
df_map = pd.read_csv("../data/processed/permno_cik_map.csv")
print(df_map.shape)
df_map.head()

(4921, 3)


,tic,cik,permno
0,aapl,320193,14593
1,msft,789019,10107
2,unh,731766,92655
3,jnj,200406,22111
4,v,1403161,46842


Merge map to similarity scores:

In [20]:
df_similarity_permno = df_score.merge(df_map, on="cik", how="left")
print(df_similarity_permno.shape)
df_similarity_permno.head()


(1303803, 5)


,cik,filing_date,similarity,tic,permno
0,1750,1994-01-13,NaN,air,54594.0
1,1750,1994-04-14,0.991912,air,54594.0
2,1750,1994-08-24,0.934598,air,54594.0
3,1750,1994-10-14,0.928699,air,54594.0
4,1750,1995-01-13,0.868299,air,54594.0


### Aligning filing dates to stock return dates 

Stock returns (df_returns) have monthly dates (e.g., '2012-05-31'), but filing dates are specific days (e.g., '2012-05-07').

- Solution: convert each filing_date to the correct monthly date. We'll assume we want the filing's filing month, so we set filing date to the end of the month.

In [21]:
df_similarity_permno['month'] = df_similarity_permno['filing_date'].dt.to_period('M').dt.to_timestamp('M')

df_similarity_permno.head()

,cik,filing_date,similarity,tic,permno,month
0,1750,1994-01-13,NaN,air,54594.0,1994-01-31
1,1750,1994-04-14,0.991912,air,54594.0,1994-04-30
2,1750,1994-08-24,0.934598,air,54594.0,1994-08-31
3,1750,1994-10-14,0.928699,air,54594.0,1994-10-31
4,1750,1995-01-13,0.868299,air,54594.0,1995-01-31


### Merge scores with returns:

In [22]:
df_merge = df_similarity_permno.merge(
    df_returns, 
    left_on=["permno", "month"], 
    right_on=["permno", "date"],
    how="left"
)
print(df_merge.shape)
df_merge.head()


(1303803, 8)


,cik,filing_date,similarity,tic,permno,month,date,ret
0,1750,1994-01-13,NaN,air,54594.0,1994-01-31,1994-01-31,14.6207
1,1750,1994-04-14,0.991912,air,54594.0,1994-04-30,NaT,NaN
2,1750,1994-08-24,0.934598,air,54594.0,1994-08-31,1994-08-31,-2.7027
3,1750,1994-10-14,0.928699,air,54594.0,1994-10-31,1994-10-31,-4.8462
4,1750,1995-01-13,0.868299,air,54594.0,1995-01-31,1995-01-31,1.8318


Cleaning it up:

In [24]:
df_clean = df_merge[['cik', 'tic', 'month', 'similarity', 'ret']]
print(df_clean.shape)
df_clean.head()


(1303803, 5)


,cik,tic,month,similarity,ret
0,1750,air,1994-01-31,NaN,14.6207
1,1750,air,1994-04-30,0.991912,NaN
2,1750,air,1994-08-31,0.934598,-2.7027
3,1750,air,1994-10-31,0.928699,-4.8462
4,1750,air,1995-01-31,0.868299,1.8318


Saving the file for future portfolio construction and analysis:

In [25]:
# Save to processed folder
output_path = "../data/processed/full_similarity_with_returns.csv"
df_clean.to_csv(output_path, index=False)

print(f"Saved cleaned similarity + returns file with shape {df_clean.shape} to {output_path}")


Saved cleaned similarity + returns file with shape (1303803, 5) to ../data/processed/full_similarity_with_returns.csv
